In [1]:
import cv2
import os
from dotenv import load_dotenv
from tqdm import tqdm
import numpy as np
import pandas as pd

In [2]:
load_dotenv()
DATA_PATH = os.path.join(os.getcwd(), os.getenv('DATA_PATH'))

In [3]:
imgs = []
labels = []

for folder in os.listdir(DATA_PATH):
    for i, img in enumerate(os.listdir(os.path.join(DATA_PATH, folder))):
        img = cv2.imread(os.path.join(DATA_PATH, folder, img))
        imgs.append(img)
        labels.append(folder)

In [4]:
print(f'Total gambar: {len(imgs)}\nJumlah kelas: {len(os.listdir(DATA_PATH))}')
print('\n')
print(f'Jumlah gambar pada masing-masing kelas\n-----------------------')
for i, label in enumerate(os.listdir(DATA_PATH)):
    for _,_,files in os.walk(os.path.join(DATA_PATH, label)):
        print(i+1, label.split('___')[1], '--->', len(files), 'gambar')

Total gambar: 6844
Jumlah kelas: 4


Jumlah gambar pada masing-masing kelas
-----------------------
1 healthy ---> 1859 gambar
2 Common_rust_ ---> 1907 gambar
3 Northern_Leaf_Blight ---> 1436 gambar
4 Gray_leaf_spot ---> 1642 gambar


# Komputasi Fitur

In [5]:
from glcm import calc_glcm_all_agls

def calculate_feature(images, labels):
    # Kalkulasi GLCM
    properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']
    features = []
    for img, label in zip(images, labels):
        combined_features = []
        glcm_features = calc_glcm_all_agls(img, label, props=properties)
        features.append(glcm_features)

    columns = []
    angles = ['0', '45', '90', '135']
    distances = [1, 2, 3, 4, 5]

    for distance in distances:
        for name in properties:
            for ang in angles:
                columns.append(f"{name}_{distance}_{ang}")

    columns.append("label")
    
    return features, columns

In [14]:
features, columns = calculate_feature(imgs, labels)

In [17]:
glcm_features = pd.DataFrame(features, columns=columns)
glcm_features.to_csv('glcm_features.csv')